In [1]:
import difflib
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sys
sys.path.insert(0, '..')
import warnings
import zlib

# local
from libs.etl.data_cleaning import generate_aep_outage_dfs, generate_duke_outage_dfs, generate_iir_boilers_dfs, determine_component
from libs.inspection_analysis.utils import analyze_inspection_df, hist_inspection_df, check_thickness, get_thickness_histogram, group_critdat, get_inspection_df
from libs.interfaces.rolodex_interface import rolodex_load
from libs.reference.constants import COMPONENT_TYPE_MAP, POWER_CUST_LIST
from libs.viz.utils import add_vbar_labels, add_hbar_labels

In [2]:
plt.style.use('dark_background')
#sns.set_style('darkgrid')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

warnings.filterwarnings('ignore')

wd = os.getcwd()
print(wd)

C:\Users\tom.fiorelli\Documents\GitHub\davos-data-project\notebooks


In [3]:
us_boilers_df, operational_us_boilers_df = generate_iir_boilers_dfs()

In [4]:
data = rolodex_load(
    "inspection", 
    links=[
        "component", 
        "unit", 
        "site", 
        "organization"
    ], 
    filters=[
        {
            "type": "component", 
            "key": "portal.component_type", 
            "value": "1"
        },
        {
            "type": "inspection",
            "key": "portal.status",
            "value": "completed"
        },
        {
            "type": "inspection",
            "key": "portal.inspection_type",
            "value": "wall_thickness"
        }
    ], 
    ids=[], 
    tags={
        "organization": "aes-ea4f30"
    }, 
    limit=100000
)

In [5]:
aes_boiler_inspections_df = pd.json_normalize(data)

In [6]:
aes_boiler_inspections_df.shape

(56, 129)

In [7]:
aes_boiler_inspections_df.head()

,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,links.organization.tags.organization,links.unit.data.radar.color,links.unit.data.geometry.type,links.unit.data.geometry.coordinates,links.unit.data.asset_type,links.unit.data.fde_analysis,links.unit.data.gecko_customer,links.site.data.values.fde_analysis,links.site.data.fde_analysis,links.component.data.display_name,links.unit.data.opacity,links.unit.data.values.color

In [8]:
aes_boiler_inspections_df[aes_boiler_inspections_df['links.component.data.portal.wall_nominal_thickness'].isna()]

,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,links.organization.tags.organization,links.unit.data.radar.color,links.unit.data.geometry.type,links.unit.data.geometry.coordinates,links.unit.data.asset_type,links.unit.data.fde_analysis,links.unit.data.gecko_customer,links.site.data.values.fde_analysis,links.site.data.fde_analysis,links.component.data.display_name,links.unit.data.opacity,links.unit.data.values.color

20200309-f4ab1f - waterwalls-745c2a - 0.165
20201030-f78847 - waterwalls-aa77e4 - 0.24

20201106-40276d - division-wall-4b5a3f - 0.25
20201106-afc7e0 - waterwalls-bfc123 - 0.22 (.2 in section of front wall)

20201110-50c594 - division-wall-4eca7c - 0.25
20201110-5b37fd - waterwalls-89ed80 - 0.22 (.2 in section of front wall)

In [9]:
# division walls are commented out here

na_filler_df = pd.DataFrame({
    "slug": [
        "20200309-f4ab1f",
        "20201030-f78847",
#        "20201106-40276d",
        "20201106-afc7e0",
#        "20201110-50c594",
        "20201110-5b37fd"
    ],
    "nominal": [
        0.165,
        0.24,
#        0.25,
        0.2,
#        0.25,
        0.2
    ]
})

In [10]:
aes_boiler_inspections_df['Component_type'] = aes_boiler_inspections_df['entry.tags.component'].astype(str).apply(lambda x: determine_component(x))

In [11]:
aes_boiler_inspections_df.groupby(['Component_type'])['entry.id'].nunique().to_frame()

,entry.id
Component_type,
Backpass,11
Bullnose,1
Division Wall,2
Slope,5
Steamdrum,2
Wall,18
boiler,2
front,1
horizontal-sh-panels-1-99,1


In [12]:
## NOTE: dropping the manway and backpass inspections
analysis_list = [
    'Wall', 
    'Slope', 
    'boiler', 
    'front', 
    'pc-boiler-1', 
    'Backpass'
]

In [13]:
filtered_aes_boiler_inspections_df = aes_boiler_inspections_df[aes_boiler_inspections_df['Component_type'].isin(analysis_list)]

In [14]:
filtered_aes_boiler_inspections_df.shape

(38, 130)

In [15]:
filtered_aes_boiler_inspections_df.loc[:, (
    'entry.tags.organization',
    'entry.tags.site',
    'entry.tags.unit',
    'entry.tags.component'
)].sort_values(by=[
    'entry.tags.organization',
    'entry.tags.site',
    'entry.tags.unit',
    'entry.tags.component'
])

,entry.tags.organization,entry.tags.site,entry.tags.unit,entry.tags.component
51,aes-ea4f30,aes-gener-electrica-angamos-42c53f,unit-1-eb5fe4,slopes-d11630
44,aes-ea4f30,aes-gener-electrica-angamos-42c53f,unit-1-eb5fe4,waterwalls-a55be1
31,aes-ea4f30,harding-street-generating-station-abae5f,unit-7-2f3456,slopes-2cfc14
30,aes-ea4f30,harding-street-generating-station-abae5f,unit-7-2f3456,waterwalls-4c9846
17,aes-ea4f30,hawaii-a56ac8,boiler-a-934982,waterwalls-2e3bab
27,aes-ea4f30,hawaii-a56ac8,boiler-b-2a16ff,waterwalls-6923b2
36,aes-ea4f30,huntington-beach-bb1307,unit-2-389961,waterwalls-aa77e4
7,aes-ea4f30,petersburg-b4d346,boiler-bb7dcf,waterwalls-c2a87e
28,aes-ea4f30,petersburg-b4d346,boiler-u2-774f1a,waterwalls-c6a00f
52,aes-ea4f30,petersburg-b4d346,unit-3-e43a85,lower-slopes-c0de19


In [16]:
sliced_inspection_df = filtered_aes_boiler_inspections_df.loc[:, ('entry.data.portal.slug', 'links.component.data.portal.wall_nominal_thickness')].reset_index()

sliced_inspection_df.rename(
    columns={
        'entry.data.portal.slug': 'slug',	
        'links.component.data.portal.wall_nominal_thickness': 'nominal',
    }, 
    inplace=True
)

sliced_inspection_df.drop(columns={'index'}, inplace=True)

In [17]:
sliced_inspection_df.shape

(38, 2)

In [18]:
set(sliced_inspection_df.nominal)

{0.165,
 0.195,
 0.206,
 0.22,
 0.222,
 0.229,
 0.24,
 0.25,
 0.26,
 0.28,
 0.3,
 nan,
 nan,
 nan}

In [19]:
clean_nominals_df = sliced_inspection_df[
    ~sliced_inspection_df['nominal'].isna() &
    (sliced_inspection_df['nominal'] !=0) &
    (sliced_inspection_df['nominal'] < 1) &
    (sliced_inspection_df['nominal'] > .1)
]

In [20]:
clean_nominals_df.shape

(35, 2)

In [21]:
clean_nominals_df = pd.concat([clean_nominals_df, na_filler_df])

In [22]:
clean_nominals_df.shape

(39, 2)

In [23]:
clean_nominals_df.reset_index(inplace=True)

In [24]:
histogram_df, hist_error_slug_list = get_thickness_histogram(clean_nominals_df)

0/39
Error: No plots record on req.json ...
Error: No plots record on req.json ...
10/39
20/39
30/39
Error: No plots record on req.json ...


In [25]:
histogram_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss
0,20180416-12319e,0.260,0.152,0.325,162,32634,2,93,29,12,3,0,0,0,0,0,478,52,22,6,0,0,0,0,0
1,20180416-2118b9,0.195,0.126,0.197,26,288,0,25,12,6,0,0,0,0,0,0,116,19,7,0,0,0,0,0,0
2,20180416-abc541,0.195,0.116,0.200,26,289,35,26,23,10,1,0,0,0,0,0,147,49,14,1,0,0,0,0,0
3,20180926-254609,0.195,0.101,0.205,26,297,67,26,19,9,2,0,0,0,0,0,124,45,15,2,0,0,0,0,0
4,20180926-ca61af,0.195,0.121,0.206,26,306,0,25,14,2,0,0,0,0,0,0,76,21,2,0,0,0,0,0,0


In [26]:
histogram_df.shape

(36, 25)

In [27]:
comb_df = histogram_df.merge(filtered_aes_boiler_inspections_df, how='left', left_on='slug', right_on='entry.data.portal.slug')

In [28]:
comb_df.shape

(36, 155)

In [29]:
len(hist_error_slug_list)

3

In [30]:
hist_error_slug_list

['20170425-33a954', '20170427-4b31bf', '20220509-27e470']

In [31]:
aes_boiler_inspections_df[aes_boiler_inspections_df['entry.data.portal.slug'].isin(hist_error_slug_list)]

,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,links.organization.tags.organization,links.unit.data.radar.color,links.unit.data.geometry.type,links.unit.data.geometry.coordinates,links.unit.data.asset_type,links.unit.data.fde_analysis,links.unit.data.gecko_customer,links.site.data.values.fde_analysis,links.site.data.fde_analysis,links.component.data.display_name,links.unit.data.opacity,links.unit.data.values.color

In [32]:
aes_boiler_inspections_df.shape

(56, 130)

In [33]:
aes_dupeless = comb_df.sort_values(by=['entry.data.portal.date'], ascending=False).drop_duplicates(subset=['entry.tags.unit', 'entry.tags.component'], keep='first')

In [34]:
aes_dupeless.groupby(['Component_type'])['Bins w 30% Loss'].sum().to_frame()

,Bins w 30% Loss
Component_type,
Backpass,285
Slope,613
Wall,1057
boiler,673
front,0
pc-boiler-1,9


In [35]:
aes_dupeless.groupby(['Component_type'])['Bins w 30% Loss'].mean().to_frame()

,Bins w 30% Loss
Component_type,
Backpass,71.2500
Slope,122.6000
Wall,66.0625
boiler,673.0000
front,0.0000
pc-boiler-1,9.0000


In [36]:
comb_df['entry.tags.unit'].nunique()

19

In [37]:
comb_df.sort_values(by=['Bins w 30% Loss'], ascending=False).head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,link

In [38]:
IIR_ID_all_df = pd.read_csv('../data/outputs/mapped_us_boilers_12523.csv')

IIR_unit_map_df = IIR_ID_all_df.loc[:,('IIR_UNIT_ID', 'Unit.slug')].drop_duplicates()
IIR_unit_map_df.shape

iir_mapped_comb_df = comb_df.merge(IIR_unit_map_df, how='left', left_on='entry.tags.unit', right_on='Unit.slug')

In [39]:
us_boilers_df, operational_us_boilers_df = generate_iir_boilers_dfs()

In [40]:
enriched_boiler_inspections_df = iir_mapped_comb_df.merge(us_boilers_df, how='left', left_on='IIR_UNIT_ID', right_on='UNIT_ID')

In [41]:
enriched_boiler_inspections_df.shape

(38, 210)

In [42]:
enriched_boiler_inspections_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,link

In [43]:
enriched_boiler_inspections_df.groupby(['FUEL_GROUP'])['entry.tags.unit'].nunique()

FUEL_GROUP
Coal           7
Natural Gas    3
Name: entry.tags.unit, dtype: int64

In [44]:
enriched_boiler_inspections_df[enriched_boiler_inspections_df['FUEL_GROUP']=='Natural Gas']

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,link

In [45]:
enriched_boiler_inspections_df[enriched_boiler_inspections_df['FUEL_GROUP']=='Coal']

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,link

In [46]:
enriched_boiler_inspections_df[enriched_boiler_inspections_df['Component_type'].isin(['Slope', 'Wall'])].groupby(['FUEL_GROUP'])['crits_per10k'].median().to_frame()

,crits_per10k
FUEL_GROUP,
Coal,7.0
Natural Gas,1.5


In [47]:
1.5/7

0.21428571428571427

In [48]:
aes_recent_inspection_df = aes_dupeless.groupby(['entry.tags.site', 'links.unit.data.portal.name', 'Component_type'])['bins_collected', 'Bins w 30% Loss', 'Bins w 40% Loss', 'Bins w 50% Loss', 'Bins w 60% Loss'].sum()aes_recent_inspection_df = aes_dupeless.groupby(['entry.tags.site', 'links.unit.data.portal.name', 'Component_type'])['bins_collected', 'Bins w 30% Loss', 'Bins w 40% Loss', 'Bins w 50% Loss', 'Bins w 60% Loss'].sum()

SyntaxError: invalid syntax (115893591.py, line 1)

In [ ]:
.aes_recent_inspection_dfto_csv('../data/outputs/aes_recent_inspections_data.csv')

In [49]:
aes_recent_inspection_df = aes_dupeless.groupby(['entry.tags.site', 'links.unit.data.portal.name', 'Component_type'])['nominal', 'bins_collected', 'Bins w 10% Loss', 'Bins w 20% Loss', 'Bins w 30% Loss', 'Bins w 40% Loss'].sum()

In [50]:
aes_recent_inspection_df

nominal  \
entry.tags.site                          links.unit.data.portal.name Component_type            
aes-gener-electrica-angamos-42c53f       Unit 2                      Slope             0.250   
                                                                     Wall              0.250   
harding-street-generating-station-abae5f Unit 7                      Slope             0.240   
                                                                     Wall              0.240   
hawaii-a56ac8                            Unit A                      Wall              0.280   
                                         Unit B                      Wall              0.280   
huntington-beach-bb1307                  Unit 2                      Wall              0.240   
petersburg-b4d346                        Boiler U1                   Wall              0.220   
                                         Boiler U3                   Slope             0.220   
                                                                     Wall              0.220   
                                         Boiler U4                   Slope             0.222   
                                                                     boiler            0.222   
                                         Unit 2                      Wall              0.206   
plant-itabo-48f6b4                       PC Boiler                   Slope             0.229   
                                                                     pc-boiler-1       0.229   
southland-6eb6a2                         Unit 5                      Wall              0.280   
                                         Unit 6                      Wall              0.280   
southland-alamitos-632170                Unit 3                      Wall              0.200   
                                         Unit 4                      Wall              0.200   
tamuin-termoelectrica-del-golfo-dbc457   Boiler 1B                   Wall              0.165   
                                         CFB Boiler 1A               Wall              0.165   
ventanas-c1a523                          Unit 2                      Wall              0.250   
                                         Unit 3                      front             0.300   
warrior-run-134143                       Boiler 1                    Backpass          0.780   
                                                                     Wall              0.260   

                                                                                     bins_collected  \
entry.tags.site                          links.unit.data.portal.name Component_type                   
aes-gener-electrica-angamos-42c53f       Unit 2                      Slope                     5785   
                                                                     Wall                     51411   
harding-street-generating-station-abae5f Unit 7                      Slope                    12588   
                                                                     Wall                     70706   
hawaii-a56ac8                            Unit A                      Wall                     32593   
                                         Unit B                      Wall                     32131   
huntington-beach-bb1307                  Unit 2                      Wall                     42571   
petersburg-b4d346                        Boiler U1                   Wall                     50400   
                                         Boiler U3                   Slope                    20916   
                                                                     Wall                    106229   
                                         Boiler U4                   Slope                    22849   
                                                                     boiler                   94567   
                                         Unit 2   

In [51]:
enriched_boiler_inspections_df[enriched_boiler_inspections_df['entry.tags.site']=='tamuin-termoelectrica-del-golfo-dbc457']

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,Tubes w 40% Loss,Tubes w 50% Loss,Tubes w 60% Loss,Tubes w 70% Loss,Tubes w 80% Loss,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss,entry.id,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,entry.data.portal.show_excel,entry.data.portal.sort_order,entry.data.portal.data_import,entry.data.portal.external_id,entry.data.portal.zero_offset,entry.data.portal.component_id,entry.data.portal.inspection_type,entry.data.portal.ndt_method_type,entry.data.portal.target_release_time,entry.data.portal.sales_robotic_service_id,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.inspection,entry.tags.organization,links.component.id,links.component.type,links.component.data.portal.id,links.component.data.portal.lat,links.component.data.portal.lng,links.component.data.portal.name,links.component.data.portal.slug,links.component.data.portal.unit_id,links.component.data.portal.alloy_id,links.component.data.portal.contents,links.component.data.portal.sea_level,links.component.data.portal.description,links.component.data.portal.slope_angle,links.component.data.portal.temperature,links.component.data.portal.tube_spacing,links.component.data.portal.material_type,links.component.data.portal.salesforce_id,links.component.data.portal.tube_diameter,links.component.data.portal.component_type,links.component.data.portal.shell_diameter,links.component.data.portal.sales_asset_type_id,links.component.data.portal.wall_nominal_thickness,links.component.data.portal.wall_warning_thickness,links.component.data.portal.coating_nominal_thickness,links.component.data.portal.coating_warning_thickness,links.component.data.portal.difficulty_factor_percentage,links.component.tags.site,links.component.tags.unit,links.component.tags.component,links.component.tags.organization,links.unit.id,links.unit.type,links.unit.data.portal.id,links.unit.data.portal.name,links.unit.data.portal.slug,links.unit.data.portal.capacity,links.unit.data.portal.latitude,links.unit.data.portal.fuel_type,links.unit.data.portal.longitude,links.unit.data.portal.built_date,links.unit.data.portal.description,links.unit.data.portal.location_id,links.unit.data.portal.total_tubes,links.unit.data.portal.salesforce_id,links.unit.data.portal.inspection_code,links.unit.data.portal.retirement_date,links.unit.data.portal.sales_asset_category_id,links.unit.data.portal.inspection_interval_years,links.unit.data.display_name,links.unit.tags.site,links.unit.tags.unit,links.unit.tags.organization,links.unit.tags.radar_demo_id,links.site.id,links.site.type,links.site.data.portal.id,links.site.data.portal.lat,links.site.data.portal.lng,links.site.data.portal.city,links.site.data.portal.name,links.site.data.portal.slug,links.site.data.portal.region,links.site.data.portal.country,links.site.data.portal.customer_id,links.site.data.portal.postal_code,links.site.data.portal.industry_type,links.site.data.portal.salesforce_id,links.site.data.portal.slack_channel,links.site.data.portal.address_line_1,links.site.data.portal.address_line_2,links.site.data.portal.drive_folder_id,links.site.data.values.gecko_customer,links.site.data.geometry.type,links.site.data.geometry.coordinates,links.site.data.display_name,links.site.data.gecko_customer,links.site.tags.site,links.site.tags.organization,links.site.tags.radar_demo_id,links.organization.id,links.organization.type,links.organization.data.portal.id,links.organization.data.portal.logo,links.organization.data.portal.name,links.organization.data.portal.slug,links.organization.data.portal.salesforce_id,links.organization.data.portal.drive_folder_id,link

In [52]:
TEG_2019 = get_inspection_df('20190311-d526d9')
TEG_2020 = get_inspection_df('20200309-f4ab1f')

Making Req...
...Req Done
Making Req...
...Req Done


In [53]:
min_t, max_t, tubes_inspected, bins_collected, tube_hist_bin_counts, bin_hist_bin_counts = hist_inspection_df(TEG_2019, 5.3)

In [54]:
bin_hist_bin_counts

[11389, 135, 1, 0, 0, 0, 0, 0, 0]

In [55]:
min_t, max_t, tubes_inspected, bins_collected, tube_hist_bin_counts, bin_hist_bin_counts = hist_inspection_df(TEG_2020, 5.3)

In [56]:
bin_hist_bin_counts

[13563, 132, 3, 0, 0, 0, 0, 0, 0]